## Installing Libraries Needed

In [ ]:
!python -m pip install pandas

In [ ]:
!python -m pip install openpyxl


## Dataset Inventory Loader

In [1]:
import os
import re

base_path = r"G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
    ~$JANUARY_2018_METADATA.xlsx (metadata)
  April:
    APRIL_2018_METADATA.xlsx (metadata)
    APRIL_2018.CSV (survey)
    ~$APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018_METADATA.xlsx (metadata)
    OCTOBER_2018.CSV (survey)
  Unmatched:
    clean_jan_2018_metadata.csv (survey)

Year 2019:
  January:
    JANUARY_2019_METADATA.xlsx (metadata)
    JANUARY_2019.CSV (survey)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022_METADATA.xlsx (metadata)
    JANUARY_2022.csv (survey)
  Februa

## Load Dataset Function

In [2]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

 <H3>Sample: January 2018 Survey </H3>

In [3]:
import pandas as pd

# Load the survey sheet of January 2018 metadata
jan_2018_survey = load_dataset("2018", "January","survey")

# View the first few rows
jan_2018_survey.head()

,PUFREG,PUFPRV,PUFPRRCD,PUFHHNUM,PUFURB2K10,PUFPWGTPRV,PUFSVYMO,PUFSVYYR,PUFPSU,PUFRPL,...,PUFC33_WEEKS,PUFC34_WYNOT,PUFC35_LTLOOKW,PUFC36_AVAIL,PUFC37_WILLING,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT
0,14,1,100,1,2,124.9425,1,2018,140,32,...,,6,,,,1,52,2,,3
1,14,1,100,1,2,131.2126,1,2018,140,32,...,,,,,,,,1,01,1
2,14,1,100,1,2,142.0464,1,2018,140,32,...,,,,,,,,1,01,1
3,14,1,100,1,2,138.2958,1,2018,140,32,...,,,,,,,,,,
4,14,1,100,2,2,195.4152,1,2018,140,32,...,,,,,,,,1,41,1


## Metadata Sheet 1 Function 

<H5> Sample: January 2018 Metadata Sheet 1 (Raw) </H5>

In [4]:
# Load the first sheet of January 2018 metadata
january_2018_metadata_sheet1 = load_dataset("2018", "January", "metadata", 0)

# View the first few rows
print("=== January 2018 Metadata Sheet 1 ===")
january_2018_metadata_sheet1.head()


=== January 2018 Metadata Sheet 1 ===


,QUEST,Questionnaire,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,_IDS0,(Id Items),NaN,NaN
1,NaN,NaN,NaN,NaN,PUFREG,Region
2,NaN,NaN,NaN,NaN,PUFPRV,Province
3,NaN,NaN,NaN,NaN,PUFPRRCD,Province Recode
4,NaN,NaN,NaN,NaN,PUFHHNUM,Household Unique Sequential Number


### Reshaping Metadata Sheet 1

In [5]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [6]:
import pandas as pd

# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head()


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES


## Metadata Sheet 2 Function

<H5> Sample: January 2018 Metadata Sheet 2 (Raw)</H5>

In [7]:
# Load the second sheet of January 2018 metadata
january_2018_metadata_sheet2 = load_dataset("2018", "January", "metadata", 1)

# View the first few rows
print("=== January 2018 Metadata Sheet 2 ===")
january_2018_metadata_sheet2.head()

=== January 2018 Metadata Sheet 2 ===


,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


### Reshaping Metadata Sheet 2

In [ ]:
import os
import pandas as pd


def reshape_sheet2_robust(df):
    """
    Convert metadata Sheet 2 (the values dictionary) into a clean, long-format table.

    This function reads the sheet exactly as it appears in Excel, without:
    - Assuming any header row
    - Auto-filling missing values
    - Inferencing min/max values
    - Guessing variable names

    Sheet 2 typically has this layout:
        Column A = Variable name (only appears once per block)
        Column B = Variable description (blank except at the start of a block)
        Column C = Label for each value (required)
        Column D = Minimum value (optional)
        Column E = Maximum value (optional)
        Column F+ = Additional text or category notes (optional)

    The function processes rows in order and:
        - Carries forward the most recent non-empty variable name (Column A)
        - Carries forward the most recent non-empty description (Column B)
        - Creates one output row per value label (Column C)
        - Leaves missing min/max/additional values as 0
        - Reads extra info (Column F onward) if present

    Returns:
        A clean pandas DataFrame with columns:
            Variable
            Description
            Label
            min_value
            max_value
            additional_value
    """

    reshaped = []

    # Ensure all blanks are handled consistently
    df = df.fillna('').astype(str)

    # Initialize with the first variable and description
    current_var = df.iloc[0, 0].strip() or 'UNKNOWN_VAR'
    current_desc = df.iloc[0, 1].strip() or ''

    # Iterate row-by-row
    for idx, row in df.iterrows():
        # ---- Column A: Variable name ----
        var_candidate = row.iloc[0].strip()
        if var_candidate:
            current_var = var_candidate

        # ---- Column B: Description ----
        desc_candidate = row.iloc[1].strip()
        if desc_candidate:
            current_desc = desc_candidate

        # ---- Column C: Label (required) ----
        label = row.iloc[2].strip()
        if not label:
            continue   # skip rows without labels

        # ---- Column D/E: min/max values ----
        raw_min = row.iloc[3].strip()
        raw_max = row.iloc[4].strip()

        min_value = raw_min if raw_min else '0'
        max_value = raw_max if raw_max else '0'

        # ---- Column F+: Additional text ----
        extra = '0'  # default to 0
        if len(row) > 5:
            for j in range(5, len(row)):
                extra_candidate = row.iloc[j].strip()
                if extra_candidate:
                    extra = extra_candidate
                    break

        # ---- Append clean record ----
        reshaped.append({
            "Variable": current_var,
            "Description": current_desc,
            "Label": label,
            "min_value": min_value,
            "max_value": max_value,
            "additional_value": extra
        })

    return pd.DataFrame(reshaped)


# ============================================================
#   load_dataset()
# ============================================================
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load any dataset (survey or metadata) from the file inventory.

    • For SURVEY CSV: normal pandas.read_csv()
    • For METADATA Excel: read with no header, reshape Sheet 2 automatically
    """
    # Retrieve file information from inventory
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )

    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])

    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)

    # Metadata Excel — always read with no header
    df = pd.read_excel(file_path, sheet_name=sheet_number, header=None)

    # Automatic reshaping ONLY for metadata Sheet 2
    if sheet_number == 1:
        df = reshape_sheet2_robust(df)

    return df


Loading Reshaped Metadata Sheet 2 on January 2018

In [ ]:
clean_jan_2018 = load_dataset("2018", "January", "metadata", 1)
clean_jan_2018.head(20)


,Variable,Description,Label,min_value,max_value,additional_value
0,PUFREG_VS1,Region,National Capital Region,13,0,0
1,PUFREG_VS1,Region,Cordillera Administrative Region,14,0,0
2,PUFREG_VS1,Region,Region I - Ilocos Region,1,0,0
3,PUFREG_VS1,Region,Region II - Cagayan Valley,2,0,0
4,PUFREG_VS1,Region,Region III - Central Luzon,3,0,0
5,PUFREG_VS1,Region,Region IVA - CALABARZON,4,0,0
6,PUFREG_VS1,Region,Region IVB - MIMAROPA,17,0,0
7,PUFREG_VS1,Region,Region V- Bicol,5,0,0
8,PUFREG_VS1,Region,Region VI - Western Visayas,6,0,0
9,PUFREG_VS1,Region,Region VII - Central Visayas,7,0,0


Loading Reshaped Metadata Sheet 2 on August 2024

In [10]:
clean_aug_2024 = load_dataset("2024", "August", "metadata", 1)
clean_aug_2024.head(20)

,Variable,Description,Label,min_value,max_value,additional_value
0,PUFSVYMO_VS1,Survey Month,January,1.0,0,0
1,PUFSVYMO_VS1,Survey Month,February,2.0,0,0
2,PUFSVYMO_VS1,Survey Month,March,3.0,0,0
3,PUFSVYMO_VS1,Survey Month,April,4.0,0,0
4,PUFSVYMO_VS1,Survey Month,May,5.0,0,0
5,PUFSVYMO_VS1,Survey Month,June,6.0,0,0
6,PUFSVYMO_VS1,Survey Month,July,7.0,0,0
7,PUFSVYMO_VS1,Survey Month,August,8.0,0,0
8,PUFSVYMO_VS1,Survey Month,September,9.0,0,0
9,PUFSVYMO_VS1,Survey Month,October,10.0,0,0


Checking for Variables with Additional Value (Extra Column)

In [11]:
# Load metadata Sheet 2 for your desired month/year
df_metadata = load_dataset("2018", "January", "metadata", 1)

# Filter rows for PUFC10_CONWR_VS1
pu_fc10 = df_metadata[df_metadata["Variable"] == "PUFC10_CONWR_VS1"]

# Display the result
pu_fc10


,Variable,Description,Label,min_value,max_value,additional_value
218,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Overseas Contract Workers,1,0,0
219,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Workers other than OCW,2,0,0
220,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,"Employees in Philippine Embassy, Consulates & ...",3,0,0
221,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Students abroad/Tourists,4,0,0
222,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Others,5,0,0
223,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Less than 15 Years Old,0,0,Not Applicable


Downloading the Reshaped Metadata Sheet 2 as a CSV file

In [12]:
import os

# Load January 2018 metadata Sheet 2 and reshape
jan_2018_sheet2 = load_dataset("2018", "January", "metadata", 1)

# Specify output CSV path
output_csv_path = os.path.join(base_path, "2018", "clean_jan_2018_metadata_sheet2.csv")

# Save the entire reshaped Sheet 2 to CSV
jan_2018_sheet2.to_csv(output_csv_path, index=False)

print(f"Reshaped Sheet 2 saved to: {output_csv_path}")


Reshaped Sheet 2 saved to: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\2018\clean_jan_2018_metadata_sheet2.csv
